In [1]:
import cv2
import time
import numpy as np
from skimage import transform
import imutils
from imutils.video import VideoStream
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow import expand_dims
import os
import tensorflow as tf

In [2]:
# Load the mask detector model
mask_net = load_model('face_mask_detector_Threeclasses (1).model')

# Load the face detector model
prototxtPath = r'C:\Users\mvcc1\caffemodel\deploy.prototxt'
weightsPath = r'C:\Users\mvcc1\caffemodel\res10_300x300_ssd_iter_140000.caffemodel'

face_net = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)

font = cv2.FONT_HERSHEY_SIMPLEX

# Initialize the webcam
webcam = VideoStream(src=0).start()
time.sleep(2.0)

while True:
    frame = webcam.read()
    frame = imutils.resize(frame, width=600)
    #print(frame)
    faces = []
    locs = [] # Locations
    
    # Grab the dimensions of a frame and then construct a blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224), (104.0, 177.0, 123.0))
    # Pass the blob through the face network to get the face detections
    face_net.setInput(blob)
    face_detections = face_net.forward()
    #print(face_detections)
    # Loop over the face detections

    for i in range(0, face_detections.shape[2]):
        # Extract the confidence associated with the face detections
        confidence = face_detections[0, 0, i, 2]
        #print(confidence)
        # Filter out weak detections to ensure the confidence is greater than the minimum
        if confidence > 0.5:
            # Compute the (x, y) coordinates of the bounding box for the object
            box = face_detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype(np.int)
            # Making sure the bounding boxes fall within the dimensions of the frame
            (x, y) = (max(0, x), max(0, y))
            (x1, y1) = (min(w-1, x1), min(h-1, y1))
            try:
                # Slice the frame, convert to RGB and preprocess
                face = frame[y:y1, x:x1]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                face = cv2.resize(face, (224, 224))
                face = img_to_array(face)
                face = preprocess_input(face)
                # Expand to fit the frame
                face = expand_dims(face, axis=0)
                # Append to their respective lists
                faces.append(face)
                locs.append((x, y, x1, y1))
                # Make the predictions('incorrect_mask' 'with_mask' 'without_mask') 
                preds = mask_net.predict(faces)
                print(preds)# Making predictions on the frame
                labels = np.argmax(preds)
                print(labels)
            except:
                pass
    
    for (box, preds) in zip(locs, preds):
         #Unpack the bounding box and predictions
        (x, y, x1, y1) = box
        (wrongMask, Mask, noMask) = preds
       
        if labels == 1:
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 15), 2)
            cv2.putText(frame, 'Mask', (x, y-8), font, 0.5, (0, 255, 15), 2)
        
        elif labels == 0:
            cv2.rectangle(frame, (x, y), (x1, y1), (15, 50, 100), 2)
            cv2.putText(frame, 'Incorrect Mask', (x, y-8), font, 0.5, (15, 50, 100), 2)
            
        else:
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 0, 255), 2)
            cv2.putText(frame, 'No Mask', (x, y-8), font, 0.5, (0, 0, 255), 2)
        
    cv2.imshow('Live!', frame)
    key = cv2.waitKey(5)
    if key == 27:
        break

cv2.destroyAllWindows()
webcam.stop()

[[0.03217401 0.61502427 0.3528017 ]]
1
[[0.06889313 0.38390225 0.5472046 ]]
2
[[0.03443801 0.2488999  0.71666205]]
2
[[0.01903227 0.30186534 0.67910236]]
2
[[0.02450632 0.312078   0.6634156 ]]
2
[[0.01688034 0.25039577 0.73272383]]
2
[[0.01229876 0.2654489  0.7222523 ]]
2
[[0.00586047 0.08586125 0.9082782 ]]
2
[[0.00997313 0.10821067 0.8818162 ]]
2
[[0.02323182 0.29939657 0.6773716 ]]
2
[[0.04377361 0.38658705 0.5696393 ]]
2
[[0.0041185  0.03062563 0.96525586]]
2
[[0.05423184 0.88098055 0.06478757]]
1
[[0.04290523 0.9041857  0.05290917]]
1
[[0.0424602  0.41794392 0.53959596]]
2
[[0.02577022 0.28601918 0.68821067]]
2
[[0.0227783  0.24888314 0.72833854]]
2
[[0.05812486 0.5689385  0.37293667]]
1
[[0.02733204 0.9033514  0.06931652]]
1
[[0.01611462 0.98027843 0.00360699]]
1
[[0.02141204 0.9717057  0.0068823 ]]
1
[[0.01611922 0.96969384 0.01418695]]
1
[[0.02426064 0.9198257  0.05591367]]
1
[[0.01964358 0.9179537  0.06240281]]
1
[[0.01485372 0.43979216 0.5453541 ]]
2
[[0.01552966 0.508299   0

KeyboardInterrupt: 